install ffmpeg with `winget install ffmpeg`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from datetime import timedelta, datetime
import numpy as np

import os
import re
from glob import glob
from pydub.utils import mediainfo

from logger import *

ModuleNotFoundError: No module named 'pydub'

In [2]:
# load patsh to all audio files from wiadomości do Henryczka
DIR_WIADOMOSCI_DO_HENRYCZKA = '/Users/konrad/OneDrive/My Private/My documents/Other/Henryk/Alienacja rodzicielska materiały/Wiadomości do Henryczka'
recordings_files = glob(f'{DIR_WIADOMOSCI_DO_HENRYCZKA}/**/*.m4a', recursive=True)
logger.info(f'retrieved {len(recordings_files)} files')

2024-03-13 22:16:10 - INFO  - <module> - retrieved 463 files


In [6]:
def file_info(path: str) -> dict:
    _recording_info = {}

    try:
        _info = mediainfo(path)
        _filename = os.path.basename(path)
        _filename_regex_groups = re.search('^(.+) (\d+-\d+-\d+) (.+)\.(.+)$', _filename) # kind, date, name, extension
        _recording_kind = _filename_regex_groups[1]
        _recording_date = _filename_regex_groups[2]
        _recording_date = datetime.strptime(_recording_date, "%Y-%m-%d")
        _recording_name = _filename_regex_groups[3]
        _recording_type = _filename_regex_groups[4]
        _recording_duration = _info['duration']
    
        _recording_info = {
            'file': _filename,
            'name': _recording_name,
            'kind': _recording_kind,
            'date': _recording_date,
            'type': _recording_type,
            'duration': _recording_duration
        }
    except Exception as e:
        logger.warning(f'encountered error for {path}: {e}')
        return None
    
    return _recording_info

df = pd.DataFrame( {'file': [], 'name': [], 'kind': [], 'date': [], 'type': [], 'duration': []} )
for ix, i in enumerate(range(0,len(recordings_files))):
    printProgressBar(ix,len(recordings_files)-1)
    _row = file_info(recordings_files[i])
    if _row != None:
        df.loc[len(df)] = _row

 |██████████████████████████████████████████████████| 100.0% 


In [7]:
# save to csv
df.sort_values('date', inplace=True)
df.to_csv('henryk-stats.csv', index=False)

# References
- https://www.kaggle.com/code/robikscube/working-with-audio-in-python
- https://towardsdatascience.com/get-to-know-audio-feature-extraction-in-python-a499fdaefe42
- https://stackoverflow.com/questions/55669182/how-to-fix-filenotfounderror-winerror-2-the-system-cannot-find-the-file-speci